In [1]:
from utils import (
    wn, 
    get_normal_form, 
    my_split, 
    TitleInWn,
    is_lat
)
import json
from os import listdir
from os.path import isfile, join
from const import DAMP_OF_WIKIDATA_PATh, GOLD_WIKIDATA_DATASET, ALL_NEED_ARTICLE
from collections import defaultdict
import pickle
import pandas as pd
from tqdm import tqdm
from collections import deque
from xml.dom import minidom
from dataclasses import dataclass

In [2]:
set_senses = set([' '.join([get_normal_form(w).lower() for w in s.lemma.split()]) for s in wn.senses])

In [3]:
onlyfiles = [f for f in listdir(DAMP_OF_WIKIDATA_PATh) if isfile(join(DAMP_OF_WIKIDATA_PATh, f))]

In [6]:
count  = 0
onlyfiles = [f for f in listdir(DAMP_OF_WIKIDATA_PATh) if isfile(join(DAMP_OF_WIKIDATA_PATh, f))]
for file in onlyfiles:
    with open(f'{DAMP_OF_WIKIDATA_PATh}\\{file}', 'r', encoding='utf-8') as f:
        for _, line in enumerate(f):
            page = json.loads(line)
            if (page['id'] and int(page['id'][1:]) > 10000000) or 'ru' in page['descriptions']:
                print(page)
                count += 1
            if count  >= 10:
                break           
        if count  >= 10:
            break

{'id': 'Q31', 'pageid': 127, 'label': {'ru': 'Бельгия', 'en': 'Belgium'}, 'descriptions': {'ru': 'государство в Западной Европе', 'en': 'country in western Europe'}, 'aliases': {'en': ['Kingdom of Belgium', 'BEL', 'be', '🇧🇪', 'BE', 'Koninkrijk België', 'Royaume de Belgique', 'Königreich Belgien', 'Belgien'], 'ru': ['Belgique', 'Королевство Бельгия']}, 'sitelinks': {'enwiki': 'Belgium', 'ruwiki': 'Бельгия'}, 'rels': [{'rel_id': 'Q3624078', 'rank': 'preferred', 'type': 'instance_of'}, {'rel_id': 'Q43702', 'rank': 'normal', 'type': 'instance_of'}, {'rel_id': 'Q6256', 'rank': 'preferred', 'type': 'instance_of'}, {'rel_id': 'Q20181813', 'rank': 'normal', 'type': 'instance_of'}, {'rel_id': 'Q185441', 'rank': 'normal', 'type': 'instance_of'}, {'rel_id': 'Q1250464', 'rank': 'normal', 'type': 'instance_of'}, {'rel_id': 'Q113489728', 'rank': 'normal', 'type': 'instance_of'}]}
{'id': 'Q8', 'pageid': 134, 'label': {'en': 'happiness', 'ru': 'счастье'}, 'descriptions': {'en': 'mental or emotional st

In [ ]:
# with open("D:\\lbase_data\\all_straight_graph.pickle", 'wb') as f:
#     pickle.dump(straight_idx,f)
# with open("D:\\lbase_data\\all_inverse_graph.pickle", 'wb') as f:
#     pickle.dump(inverse_idx,f)

In [4]:
# with open("D:\\lbase_data\\all_straight_graph.pickle", 'rb') as f:
#     straight_idx = pickle.load(f)
# with open("D:\\lbase_data\\all_inverse_graph.pickle", 'rb') as f:
#     inverse_idx = pickle.load(f)

In [5]:
# with open("D:\\lbase_data\\clear_inverse_graph.pickle", 'wb') as f:
#     pickle.dump(clear_inverse_idx,f)
# with open("D:\\lbase_data\\clear_inverse_graph.pickle", 'rb') as f:
#     clear_inverse_idx = pickle.load(f)

In [24]:
tweets = []
to_add = set()
for file in tqdm(onlyfiles):
    with open(f'{DAMP_OF_WIKIDATA_PATh}\\{file}', 'r', encoding='utf-8') as f:
        for line in f:
            info = json.loads(line)
            if info['id'] == 'Q190':
                    print(info)
            if 'ru' in info['label'] and int(info['id'][1:]) <= 10000000 and (is_lat(info["label"]['ru'])) and 'фильм' not in info["label"]['ru']:
                try:
                    label = info["label"]['ru']
                    sense = wn.get_senses(label)
                    if info['id'] == 'Q190':
                            print(sense)        
                    if sense:
                        tweets.append((info, sense[0].lemma))
                        for elem in info['rels']:
                            to_add.add(elem['rel_id'])
                    else:
                        lemma = TitleInWn(set_senses, my_split(label).split(',')[0])
                        if info['id'] == 'Q190':
                            print(lemma, '!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!') 
                        if lemma:
                            tweets.append((info, lemma))
                            for elem in info['rels']:
                                to_add.add(elem['rel_id'])     
                except:
                    pass          

  8%|▊         | 8/104 [02:12<09:59,  6.24s/it]   

{'id': 'Q190', 'pageid': 330, 'label': {'ru': 'Бог', 'en': 'God'}, 'descriptions': {'en': 'principal object of faith in monotheistic religions, a divine entity that created and typically supervises all existence', 'ru': 'название верховного сверхъестественного существа в монотеистических религиях'}, 'aliases': {'ru': ['Творец', 'Создатель', 'Всевышний', 'Демиург', 'божество'], 'en': ['supreme being', 'G-d', 'Most High', 'Almighty', 'Creator', 'All-Knowing One', 'The Lord', 'Allah', 'Brahman', 'The All']}, 'sitelinks': {'enwiki': 'God', 'ruwiki': 'Бог'}, 'rels': [{'rel_id': 'Q23847174', 'rank': 'normal', 'type': 'instance_of'}, {'rel_id': 'Q12160552', 'rank': 'normal', 'type': 'instance_of'}, {'rel_id': 'Q635155', 'rank': 'normal', 'type': 'subclass_of'}, {'rel_id': 'Q215627', 'rank': 'normal', 'type': 'subclass_of'}, {'rel_id': 'Q18563354', 'rank': 'normal', 'type': 'subclass_of'}]}
[Sense(id="113701-N-133731", name="БОГ")]


100%|██████████| 104/104 [19:38<00:00, 11.33s/it]


In [6]:
len(tweets)

161599

In [7]:
len(to_add)

18922

In [8]:
for i in tqdm(range(5)):
    if i != 0:
        to_add = new_to_add
    print(len(to_add))
    new_to_add = set()
    for file in onlyfiles:
        with open(f'{DAMP_OF_WIKIDATA_PATh}\\{file}', 'r', encoding='utf-8') as f:
            for line in f:
                info = json.loads(line)
                if info['id'] in to_add and int(info['id'][1:]) <= 10000000 and 'ru' in info['label']  and 'фильм' not in info["label"]['ru']:
                    try:
                        label = info["label"]['ru']
                        sense = wn.get_senses(label)
                        if sense:
                            tweets.append((info, sense[0].lemma))
                        else:
                            lemma = TitleInWn(set_senses, my_split(label).split(',')[0])
                            if lemma is not None:
                                tweets.append((info, lemma))
                        for elem in info['rels']:
                            new_to_add.add(elem['rel_id'])
                    except:
                        pass
                elif info['label'] and info['id'] in to_add:
                    for elem in info['rels']:
                        new_to_add.add(elem['rel_id'])
                    tweets.append((info, info['label']['en']))

  0%|          | 0/5 [00:00<?, ?it/s]

18922


 20%|██        | 1/5 [06:59<27:57, 419.36s/it]

12391


 40%|████      | 2/5 [13:48<20:40, 413.34s/it]

...

 60%|██████    | 3/5 [20:40<13:45, 412.65s/it]

5883


 80%|████████  | 4/5 [27:22<06:48, 408.42s/it]

4331


100%|██████████| 5/5 [34:05<00:00, 409.18s/it]


In [9]:
len(tweets)

188039

In [10]:
# with open(ALL_NEED_ARTICLE, 'wb') as f:
#     pickle.dump(tweets, f)

In [4]:
data = pd.read_csv(GOLD_WIKIDATA_DATASET)
data

,Unnamed: 0.1,Unnamed: 0,synset_id,wiki_title,annotation,wiki_title_gold,WikiDataGoldTitle,WikiDataGoldId
0,0,0,115195-N,Облава,Да,Облава,Облава,Q494497
1,1,1,128158-N,Шаньдун,Да,Шаньдун,Шаньдун,Q43407
2,2,2,143699-N,Оксюморон,Да,Оксюморон,оксюморон,Q169429
3,3,3,4061-N,Доверенность,Да,Доверенность,Доверенность,Q159733
4,4,5,136544-N,Шахматная доска,Да,Шахматная доска,шахматная доска,Q110079
...,...,...,...,...,...,...,...,...
1725,1725,1993,113701-N,Царю небесный,Нет,Бог,Бог,Q190
1726,1726,1994,124807-N,Наставничество,Да,Наставничество,наставничество,Q967647
1727,1727,1995,9199-N,Трансурановые элементы,Да,Трансурановые элементы,трансурановые элементы,Q106844
1728,1728,1996,103410-N,Эсватини,Да,Эсватини,Эсватини,Q1050


In [5]:
with open(ALL_NEED_ARTICLE, 'rb') as f:
    articles = pickle.load(f)
len(articles)

188039

In [6]:
graph_path_straight = defaultdict(set)
graph_path_inverse = defaultdict(set)
id_artircle2idx_article = {}
idx_article2id_artircle = {}
for idx, (article, _) in tqdm(enumerate(articles)):
    id_artircle2idx_article[article['id']] = idx
    idx_article2id_artircle[idx] = article['id']
    for link in article['rels']:
        graph_path_straight[link['rel_id']].add(article['id'])
        graph_path_inverse[article['id']].add(link['rel_id'])

188039it [00:00, 480919.29it/s]


In [7]:
def find_reachable_vertices(graph, start_vertex):
    visited = set()
    queue = deque([start_vertex])
    reachable_vertices = set()

    while queue:
        current_vertex = queue.popleft()
        visited.add(current_vertex)
        reachable_vertices.add(current_vertex)

        for neighbor in graph[current_vertex]:
            if neighbor not in visited:
                queue.append(neighbor)

    return reachable_vertices

In [8]:
print(find_reachable_vertices(graph_path_inverse, 'Q4568'))

{'Q43229', 'Q96116695', 'Q23958852', 'Q99527517', 'Q12973014', 'Q16334298', 'Q104093226', 'Q35120', 'Q96251598', 'Q104086571', 'Q327245', 'Q1639378', 'Q106559804', 'Q16334295', 'Q7184903', 'Q15720476', 'Q24017414', 'Q58778', 'Q488383', 'Q4568', 'Q4438121'}


In [9]:
def extractCtxS(wn, lemma:str):
    #составляем контекст для слова из wordnet
    ctx_s = set()
    #synonymy
    for sense in wn.get_synsets(lemma):
        for synonymy in sense.senses:
            ctx_s.update(my_split(synonymy.lemma).split(","))
    #Hypernymy/Hyponymy
    for sense in wn.get_senses(lemma):
        for hypernyms in sense.synset.hypernyms:
            ctx_s.update(my_split(hypernyms.title).split(","))
    for sense in wn.get_senses(lemma):
        for hyponyms in sense.synset.hyponyms:
            ctx_s.update(my_split(hyponyms.title).split(","))
    #Sisterhood:
    for sense in wn.get_senses(lemma):
        for hypernyms in sense.synset.hypernyms:
            for sister in hypernyms.hyponyms:
                ctx_s.update(my_split(sister.title).split(","))
    return ctx_s
@dataclass
class WnCtx:
    id: int
    ctx: set
    lemmaInWn: str
    name: str

In [10]:
file = open("D:\\asd\\ctxS.txt", "rb")
unpickler = pickle.Unpickler(file)
dictWn = unpickler.load()
file.close()

In [11]:
wn.get_synsets(wn['130555-N-741490'].lemma), wn['130555-N-741490']

([Synset(id="130555-N", title="ЯПОНСКАЯ ГЕЙША")],
 Sense(id="130555-N-741490", name="ЯПОНСКАЯ ГЕЙША"))

In [12]:
from typing import Tuple, List

In [13]:
def get_new_lemma(all_synsets, title):
    title = title.lower()
    title = title.replace("—", "-")
    title = title.replace(",", "")
    if title in all_synsets:
        return title
    if "(" in title:
        text = my_split(title).split(",")
        if text[0] in all_synsets:
            return text[0]
    text = title.split(",")
    lemmatized = " ".join([get_normal_form(word).lower()
                for word in text])
    if lemmatized in all_synsets:
        return lemmatized
    if "ё" in title:
        return get_new_lemma(all_synsets, title.replace("ё","е"))
    return None

In [14]:
all_synsets = set([lem.title.lower() for lem in wn.synsets])

In [15]:

def create_dict_candidates(articles:List[Tuple[dict, str]]) -> dict:
    """
        Создание словаря из синсета в кандидаты для связывания
    """
    canidates = defaultdict(list)
    for (article, lemma) in tqdm(articles):
        if 'ru' in article['label']:
            if lemma:
                synset_base_on_lemma = wn.get_synsets(lemma)
                if synset_base_on_lemma:
                    for synset in wn.get_synsets(lemma):
                        canidates[synset.id].append((article, lemma))
            else:
                lemma_new = get_new_lemma(all_synsets, article['label']['ru'])
                if lemma_new:
                    for synset in wn.get_synsets(lemma_new):
                        canidates[synset.id].append((article, lemma_new))
    return canidates
candidates = create_dict_candidates(articles)
print(len(candidates))

100%|██████████| 188039/188039 [02:36<00:00, 1198.52it/s]

32010


In [16]:
# candidates['ЭНЦИКЛОПЕДИЧЕСКИЙ СЛОВАРЬ']
import re


def clean_text(text):
    cleaned_text = re.sub(r"[^\w\s]", "", text)
    cleaned_text = re.sub(r"\d+", "", cleaned_text)
    cleaned_text = re.sub(r"\s+", " ", cleaned_text)
    cleaned_text = cleaned_text.strip().lower()
    return cleaned_text

def extract_ctx_wikidata(article:dict) -> set:
    '''
        Излвечения контекста статьи из ее ооописания на русском
    '''
    ctx = set()
    if 'ru' in article['descriptions']:
        for token in clean_text(article['descriptions']['ru']).split():
            ctx.add(get_normal_form(token).lower())
    return ctx
def get_score(elem1:set, elem2:set) -> float:
    '''
        Функция подсчета близости статьи и контекстов на основе эвристики
    '''
    return (len((elem1 & elem2)) + 1) / (len(elem1) + len(elem2) + 1)

In [18]:
score_dict = defaultdict(list)
for i, (title_synset, candidatess) in tqdm(enumerate(candidates.items())):
    for candidate, lemma in candidatess:
        if 'N' in wn[lemma][0].id:
            synset_ctx = dictWn[wn[lemma][0].id].ctx
            article_ctx = extract_ctx_wikidata(candidate)
            article_ctx.update([candidate['label']['ru'].lower(), lemma.lower()])
            score = get_score(article_ctx, synset_ctx)
            score_dict[title_synset].append(score)
        else:
            score_dict[title_synset].append(0.0)

32010it [05:17, 100.75it/s]


In [29]:
for candidate, lemma in candidates['103410-N']:
        if 'N' in wn[lemma][0].id:
            synset_ctx = dictWn[wn[lemma][0].id].ctx
            article_ctx = extract_ctx_wikidata(candidate)
            article_ctx.update([candidate['label']['ru'], lemma])
            score = get_score(article_ctx, synset_ctx)
            print(candidate['id'],article_ctx)
            score_dict[title_synset].append(score)
        else:
            score_dict[title_synset].append(0.0)

Q29064418 {'СВАЗИЛЕНД', 'Свазиленд', 'значение', 'страница'}


In [20]:
@dataclass
class display_synset_to_wikidata:
    id:int
    label:int
    lemma:str
    sense_id:int
    score:float
    synset_lemma:str

In [21]:
res_score={}
for _, (title_synset, candidatess) in tqdm(enumerate(candidates.items())):
    sorted_lst = sorted(zip(candidatess, score_dict[title_synset]), key=lambda x: x[1], reverse=True)
    if sorted_lst and 'N' in title_synset:
        res_score[title_synset] = display_synset_to_wikidata(sorted_lst[0][0][0]['id'], sorted_lst[0][0][0]['label'], sorted_lst[0][0][1], wn[sorted_lst[0][0][1]][0].id, sorted_lst[0][1], title_synset)

32010it [00:18, 1725.33it/s]


In [21]:
# with open("res.pickle", 'wb') as f:
#     pickle.dump(res_score, f)

In [4]:
with open("res.pickle", 'rb') as f:
    res_score = pickle.load(f)

In [22]:
len(res_score)

26409

In [26]:

data['predict_id'] = data['synset_id'].apply(lambda x: res_score[x].id if x in res_score else 'не связан')
data


,Unnamed: 0.1,Unnamed: 0,synset_id,wiki_title,annotation,wiki_title_gold,WikiDataGoldTitle,WikiDataGoldId,predict_id
0,0,0,115195-N,Облава,Да,Облава,Облава,Q494497,Q494497
1,1,1,128158-N,Шаньдун,Да,Шаньдун,Шаньдун,Q43407,Q43407
2,2,2,143699-N,Оксюморон,Да,Оксюморон,оксюморон,Q169429,Q169429
3,3,3,4061-N,Доверенность,Да,Доверенность,Доверенность,Q159733,Q159733
4,4,5,136544-N,Шахматная доска,Да,Шахматная доска,шахматная доска,Q110079,Q110079
...,...,...,...,...,...,...,...,...,...
1725,1725,1993,113701-N,Царю небесный,Нет,Бог,Бог,Q190,Q18535424
1726,1726,1994,124807-N,Наставничество,Да,Наставничество,наставничество,Q967647,Q16515647
1727,1727,1995,9199-N,Трансурановые элементы,Да,Трансурановые элементы,трансурановые элементы,Q106844,Q106844
1728,1728,1996,103410-N,Эсватини,Да,Эсватини,Эсватини,Q1050,Q29064418


In [27]:
from sklearn.metrics import accuracy_score
accuracy_score(data[data['predict_id'] !='не связан'].predict_id.astype(str), data[data['predict_id'] !='не связан'].WikiDataGoldId.astype(str)), data[data['predict_id'] !='не связан'].shape

(0.5230677052127022, (1669, 9))

In [185]:
sum([1 if i in candidates else 0 for i in data['synset_id'].values]), wn['наставничество'][0].lemma, wn.get_synsets('НАСТАВНИЧЕСТВО')

(750, 'НАСТАВНИЧЕСТВО', [Synset(id="124807-N", title="НАСТАВНИК")])

In [30]:
id_artircle2idx_article['Q7209862']

50899

In [31]:
articles[50899]

({'id': 'Q7209862',
  'pageid': 7112694,
  'label': {'en': 'Polish Air Force checkerboard', 'ru': 'Шахматная доска'},
  'descriptions': {'en': 'Marking for Polish Air Force',
   'ru': 'Опознавательный знак ВВС Польши'},
  'aliases': {},
  'sitelinks': {'enwiki': 'Polish Air Force checkerboard',
   'ruwiki': 'Шахматная доска (опознавательный знак)'},
  'rels': []},
 'шахматный доска')

In [180]:
candidates['124807-N']

[({'id': 'Q929797',
   'pageid': 880130,
   'label': {'en': 'Mentor', 'ru': 'Ментор (спутник)'},
   'descriptions': {'en': 'class of US spy satellites'},
   'aliases': {'en': ['Advanced Orion']},
   'sitelinks': {'enwiki': 'Orion (satellite)', 'ruwiki': 'Ментор (спутник)'},
   'rels': [{'rel_id': 'Q466421', 'rank': 'normal', 'type': 'subclass_of'}]},
  'ментор')]